# Tutoriel ArcticDB - Stocker et gérer des données financières

ArcticDB est une base de données orientée colonnes, optimisée pour les séries temporelles et les données financières.  
Elle permet de stocker des DataFrames pandas directement, sans conversion, et offre des performances élevées en lecture/écriture.

**Ce que vous allez apprendre :**
1. Créer une connexion ArcticDB (stockage local avec LMDB)
2. Créer des bibliothèques (libraries)
3. Écrire et lire des données (actions boursières)
4. Mettre à jour des données (append)
5. Versionner les données
6. Filtrer et requêter les données
7. Requêter avec DuckDB (SQL sur les données ArcticDB)
8. Supprimer des données

## 1. Imports et configuration

In [ ]:
import arcticdb as adb
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

## 2. Connexion à ArcticDB

ArcticDB supporte plusieurs backends de stockage : LMDB (local), S3, Azure Blob Storage.  
Pour ce tutoriel, nous utilisons **LMDB** qui stocke les données localement sur disque.

In [ ]:
# Connexion locale avec LMDB
arctic = adb.Arctic("lmdb://arcticdb_data")
print(f"Connexion établie : {arctic}")

## 3. Créer une bibliothèque (Library)

Une **library** est l'équivalent d'une base de données ou d'un schéma.  
On y regroupe des symboles (tables) par thématique.

In [ ]:
# Créer (ou récupérer) une bibliothèque
lib = arctic.get_library("actions_fr", create_if_missing=True)

# Lister les bibliothèques existantes
print("Bibliothèques disponibles :", arctic.list_libraries())

## 4. Télécharger des données financières avec yfinance

Nous allons récupérer les cours de quelques actions pour les stocker dans ArcticDB.

In [ ]:
# Liste de tickers à télécharger
tickers = ["AAPL", "MSFT", "GOOGL"]

# Télécharger 1 mois de données pour chaque ticker
donnees = {}
for ticker in tickers:
    df = yf.download(ticker, period="1mo")
    # Aplatir les colonnes multi-niveaux de yfinance
    df.columns = df.columns.get_level_values(0)
    donnees[ticker] = df
    print(f"{ticker} : {len(df)} lignes téléchargées")

# Aperçu des données Apple
donnees["AAPL"].head()

## 5. Écrire des données dans ArcticDB

La méthode `write()` stocke un DataFrame sous un **symbole** (nom de la table).  
Chaque écriture crée automatiquement une nouvelle version.

In [ ]:
# Écrire chaque ticker comme un symbole distinct
for ticker, df in donnees.items():
    info = lib.write(ticker, df)
    print(f"Écrit : {ticker} -> version {info.version}")

# Lister les symboles dans la bibliothèque
print("\nSymboles stockés :", lib.list_symbols())

## 6. Lire des données depuis ArcticDB

La méthode `read()` retourne un objet `VersionedItem` contenant le DataFrame dans `.data`.

In [ ]:
# Lire les données Apple
result = lib.read("AAPL")

print(f"Symbole : {result.symbol}")
print(f"Version : {result.version}")
print(f"Shape   : {result.data.shape}")
print()
result.data

## 7. Ajouter des données (Append)

La méthode `append()` permet d'ajouter de nouvelles lignes à un symbole existant.  
C'est idéal pour les mises à jour quotidiennes de cours boursiers.

In [ ]:
# Simuler de nouvelles données pour demain
derniere_date = donnees["AAPL"].index[-1]
nouvelle_date = derniere_date + timedelta(days=1)

nouvelles_donnees = pd.DataFrame(
    {
        "Close": [260.50],
        "High": [262.00],
        "Low": [258.00],
        "Open": [259.00],
        "Volume": [50000000],
    },
    index=pd.DatetimeIndex([nouvelle_date], name="Date"),
)

# Ajouter au symbole existant
info = lib.append("AAPL", nouvelles_donnees)
print(f"Append effectué -> version {info.version}")

# Vérifier que la ligne a été ajoutée
df_updated = lib.read("AAPL").data
print(f"Nombre de lignes : {len(df_updated)}")
df_updated.tail(3)

## 8. Versionning des données

ArcticDB garde un historique de toutes les versions d'un symbole.  
On peut relire n'importe quelle version antérieure.

In [ ]:
# Écrire une deuxième version (écriture complète, pas un append)
df_modifie = donnees["AAPL"].copy()
df_modifie["Close"] = df_modifie["Close"] * 1.05  # Simuler +5%
lib.write("AAPL", df_modifie)

# Lister toutes les versions
versions = lib.list_versions("AAPL")
print("Versions disponibles pour AAPL :")
for v in versions:
    print(f"  Version {v['version']}")

# Lire une version spécifique (la première : version 0)
ancienne = lib.read("AAPL", as_of=0)
actuelle = lib.read("AAPL")

print(f"\nVersion 0 - Close moyen : {ancienne.data['Close'].mean():.2f}")
print(f"Dernière version - Close moyen : {actuelle.data['Close'].mean():.2f}")

## 9. Filtrage avec QueryBuilder

ArcticDB permet de filtrer les données directement au niveau du stockage,  
ce qui est beaucoup plus rapide que de charger tout le DataFrame puis filtrer avec pandas.

In [ ]:
from arcticdb import QueryBuilder

# Filtrer : uniquement les jours où le volume dépasse 45 millions
q = QueryBuilder()
q = q[q["Volume"] > 45_000_000]

resultat = lib.read("AAPL", as_of=0, query_builder=q)
print("Jours avec volume > 45M :")
resultat.data

### Filtrage par plage de dates (Date Range)

On peut aussi limiter la lecture à une plage de dates spécifique  
en utilisant le paramètre `date_range`.

In [ ]:
# Lire uniquement les 5 premiers jours
df_original = lib.read("AAPL", as_of=0).data
date_debut = df_original.index[0]
date_fin = df_original.index[4] if len(df_original) > 4 else df_original.index[-1]

resultat_dates = lib.read(
    "AAPL",
    as_of=0,
    date_range=(date_debut, date_fin),
)
print(f"Données du {date_debut.date()} au {date_fin.date()} :")
resultat_dates.data

## 10. Requêter avec DuckDB

DuckDB peut requêter directement des DataFrames pandas en mémoire avec du SQL.  
Combiné à ArcticDB, cela permet de :
- **Stocker** les données avec ArcticDB (versioning, append, stockage optimisé)
- **Analyser** les données avec DuckDB (SQL complet, agrégations, jointures)

C'est le meilleur des deux mondes : stockage spécialisé + requêtes SQL puissantes.

In [ ]:
import duckdb

# Charger les données depuis ArcticDB dans des DataFrames pandas
aapl = lib.read("AAPL", as_of=0).data
msft = lib.read("MSFT").data

# DuckDB peut requêter directement les variables pandas !
# Requête simple : les 5 jours avec le plus gros volume pour Apple
duckdb.sql("""
    SELECT Date, Close, Volume
    FROM aapl
    ORDER BY Volume DESC
    LIMIT 5
""").show()

### Agrégations SQL

In [ ]:
# Statistiques agrégées
duckdb.sql("""
    SELECT
        round(avg(Close), 2)  AS close_moyen,
        round(min(Low), 2)    AS plus_bas,
        round(max(High), 2)   AS plus_haut,
        sum(Volume)           AS volume_total,
        count(*)              AS nb_jours
    FROM aapl
""").show()

### Jointures entre symboles

DuckDB permet de croiser facilement les données de plusieurs actions.

In [ ]:
# Comparer Apple et Microsoft jour par jour
duckdb.sql("""
    SELECT
        a.Date,
        round(a.Close, 2) AS aapl_close,
        round(m.Close, 2) AS msft_close,
        round(a.Close - m.Close, 2) AS ecart,
        CASE WHEN a.Volume > m.Volume THEN 'AAPL' ELSE 'MSFT' END AS plus_echange
    FROM aapl a
    JOIN msft m ON a.Date = m.Date
    ORDER BY a.Date
""").show()

### Résultat SQL vers pandas

On peut aussi récupérer le résultat DuckDB directement en DataFrame pandas pour continuer l'analyse.

In [ ]:
# Convertir le résultat SQL en DataFrame pandas
df_rendements = duckdb.sql("""
    SELECT
        Date,
        Close,
        round((Close - lag(Close) OVER (ORDER BY Date)) / lag(Close) OVER (ORDER BY Date) * 100, 2)
            AS rendement_pct
    FROM aapl
    ORDER BY Date
""").df()

# Le résultat est un DataFrame pandas classique
print(type(df_rendements))
df_rendements.dropna()

## 11. Métadonnées

On peut associer des métadonnées (dictionnaire Python) à chaque symbole.  
Utile pour stocker la source, la devise, la fréquence, etc.

In [ ]:
# Écrire avec des métadonnées
metadata = {
    "source": "yfinance",
    "devise": "USD",
    "frequence": "journalier",
    "description": "Cours Apple Inc.",
}

lib.write("AAPL", donnees["AAPL"], metadata=metadata)

# Relire les métadonnées
result = lib.read("AAPL")
print("Métadonnées associées :")
for cle, valeur in result.metadata.items():
    print(f"  {cle}: {valeur}")

## 12. Suppression

On peut supprimer un symbole spécifique ou vider une bibliothèque entière.

In [ ]:
# Supprimer un symbole
lib.delete("GOOGL")
print("Après suppression de GOOGL :", lib.list_symbols())

# Supprimer une bibliothèque entière
# arctic.delete_library("actions_fr")
# print("Bibliothèques restantes :", arctic.list_libraries())

## 13. Résumé des commandes principales

| Opération | Méthode | Description |
|-----------|---------|-------------|
| Connexion | `adb.Arctic("lmdb://chemin")` | Se connecter au stockage |
| Créer library | `arctic.get_library(nom, create_if_missing=True)` | Créer/récupérer une bibliothèque |
| Écrire | `lib.write(symbole, df)` | Stocker un DataFrame |
| Lire | `lib.read(symbole)` | Lire les données |
| Ajouter | `lib.append(symbole, df)` | Ajouter des lignes |
| Versions | `lib.list_versions(symbole)` | Lister les versions |
| Lire version | `lib.read(symbole, as_of=n)` | Lire une version spécifique |
| Filtrer | `lib.read(symbole, query_builder=q)` | Filtrer avec QueryBuilder |
| SQL (DuckDB) | `duckdb.sql("SELECT ... FROM df")` | Requêtes SQL sur les DataFrames |
| SQL -> pandas | `duckdb.sql("...").df()` | Résultat SQL en DataFrame |
| Supprimer | `lib.delete(symbole)` | Supprimer un symbole |
| Lister | `lib.list_symbols()` | Lister tous les symboles |

## Pour aller plus loin

- **Stockage S3** : `adb.Arctic("s3://bucket/chemin?region=eu-west-1")` pour un stockage cloud
- **Performances** : ArcticDB gère des milliards de lignes efficacement grâce au stockage colonnaire
- **Documentation** : [docs.arcticdb.io](https://docs.arcticdb.io)